# Vehicle License Plate Detection usingYOLOv8

- ID: N01555914
- Name: Wenhao Fang
- Group: 07

---


- The goal of this project is to learn:

  - 1. The basic steps involved in machine learning,
  - 2. Data processing methods,
  - 3. algorithms used in the YOLOv8 model,
  - 4. parameters that might affect the outcome model,
  - 5. evaluation of the machine learning model.
  - 6. a pipeline using YOLOv8.

- The outcome of this task:

  - 1. The ability to outline basic steps in machine learning.
  - 2. The ability to identify common data processing methods.
  - 3. The ability to identify algorithms in this project and point out, on a high level, why they are needed and how they work.
  - 4. The ability to configure and train model using YOLOv8.
  - 5. The ability to evaluate the outcome model.

- The expected documents by the end of this project:
  - 1. Project Report: A report of this project.
  - 2. Terminology list: A document to list terms mentioned in this project, such as algorithms, data processing methods, helpful libraries, etc.
  - 3.  Code Notebook: A Jupyter notebook to present key steps and codes.
  - 4.  Pipeline File: a Python file to define a workflow to train a YOLO model

---


## Search and Download Images

### Tool and Labrary

- `simple_image_download` :

  - a Python library that allows to search URLs of images from “Google images” using tags, and downloads them to local computer.
  - In this Assignment, Version 0.4 will be used for better compatiable.

- Command to install `simple_image_download`:

  ```sh
  pip install simple-image-download==0.4
  ```

---

### Implemenation in Python


In [1]:
# pip install simple-image-download==0.4
# Define downloady_images function
from simple_image_download import simple_image_download as simp

def download_images(keyword, num_pic):
    response = simp.simple_image_download()

    for kw in keyword:
        response.download(keywords=kw, limit=num_pic)

In [3]:
# Define a list of keywords to download images
keywords = [
    "Corvette with license plate",
    "Volkswagen Beetle with license plate",
    "Volkswagen Golf with license plate",
    "Volkswagen Passat with license plate",
    "Volkswagen Tiguan with license plate",
    "Toyota Tacoma with license plate",
    "Toyota Corolla with license plate",
    "Toyota Highlander with license plate",
    "Toyota Camry with license plate",
    "Toyota RAV4 with license plate",
    "Honda Accord with license plate",
    "Honda Pilot with license plate",
    "Honda CR-V with license plate",
    "Honda Odyssey with license plate",
    "Honda Passport with license plate",
    "Honda Civic with license plate",
    "Honda HR-V with license plate",
    "BMW with license plate",
    "Audi Q8 with license plate",
    "Audi A3 Convertible with license plate",
    "Audi A5 with license plate",
    "Audi A4 with license plate",
    "Audi Q7 with license plate",
    "Audi A8 with license plate",
    "Hyundai Kona with license plate",
    "Hyundai Sonata with license plate",
    "Hyundai Elantra with license plate",
    "Hyundai Santa Cruz with license plate",
    "Hyundai Accent with license plate",
    "Jeep Gladiator with license plate",
    "Jeep Grand Cherokee with license plate",
    "Jeep Avenger with license plate",
    "Chevrolet Camaro with license plate",
    "Chevrolet Corvette with license plate",
    "Chevrolet Malibu with license plate",
    "Ford F-Series with license plate",
    "Ford Escort with license plate",
    "Ford Fiesta with license plate",
    "Ford Focus with license plate",
    "Ford Mustang with license plate",
    "Ford Transit with license plate",
    "Ford Taurus with license plate",
    "Ford Explorer with license plate",
    "Tesla Model Y with license plate",
    "tesla model 3 with license plate",
    "tesla model s with license plate",
    "car with license plate",
    "vehicle with license plate",
    "sports car with plate",
]
# Define max number of pic to download
num_pic = 200

# Call function to dowload
# download_images(keywords, num_pic)

- Screenshot


---


## Data Processing

### Tool and Library

- `LabelImg`

  - a **graphical image annotation tool**.
  - It is written in Python and uses Qt for its **graphical interface**.
  - Annotations are saved as `XML` files in `PASCAL VOC` format, the format used by `ImageNet`. Besides, it also supports `YOLO` and `CreateML` formats.

  - Github: https://github.com/HumanSignal/labelImg

- Command to install and call programme

  ```sh
  # Install labelImg library
  pip install labelImg

  # call programme
  labelImg
  ```

  - Screenshots


---

### Split Train and Test Datasets

- Create directory for train and test datasets.
- Split images into train and test datasets.
- Copy images and annotation files to train and test directories.


In [4]:
# Define functions for data processing
import os
import shutil
from pathlib import Path
from sklearn.model_selection import train_test_split

# region Supporting Functions

def create_datasets_dir(dir):
    '''Create train and test directories'''

    if os.path.isdir(dir):
        shutil.rmtree(dir)

    Path(dir).mkdir(parents=True, exist_ok=True)

    train_path = Path(dir, "train")
    train_image_path = Path(train_path, "images")
    train_labels_path = Path(train_path, "labels")

    Path(train_path).mkdir(parents=True, exist_ok=True)
    Path(train_image_path).mkdir(parents=True, exist_ok=True)
    Path(train_labels_path).mkdir(parents=True, exist_ok=True)

    test_path = Path(dir, "test")
    test_image_path = Path(test_path, "images")
    test_labels_path = Path(test_path, "labels")

    Path(test_path).mkdir(parents=True, exist_ok=True)
    Path(test_image_path).mkdir(parents=True, exist_ok=True)
    Path(test_labels_path).mkdir(parents=True, exist_ok=True)

    test_path = Path(dir, "val")
    test_image_path = Path(test_path, "images")
    test_labels_path = Path(test_path, "labels")

    Path(test_path).mkdir(parents=True, exist_ok=True)
    Path(test_image_path).mkdir(parents=True, exist_ok=True)
    Path(test_labels_path).mkdir(parents=True, exist_ok=True)


def copy_files(source_fn_list, source_path,  target_path):
    '''Copy selected file list to destination'''

    image_path = Path(target_path,"images")
    labels_path = Path(target_path,"labels")
    for fn in source_fn_list:
        # if the csv file exists
        if Path(source_path, fn+".txt").is_file() and Path(source_path, fn+".jpeg").is_file():
            shutil.copy(Path(source_path, fn+".jpeg"), image_path)
            shutil.copy(Path(source_path, fn+".txt"), labels_path)


def report_info(target_path):
    '''report files in path'''
    report = {
        "images": {"path":Path(target_path,"images"),"count": 0},
        "labels": {"path":Path(target_path,"labels"),"count":0},
    }

    for file in os.listdir(Path(target_path,"labels")):
        if file.endswith(".txt"):
            report['labels']["count"] += 1

    for file in os.listdir(Path(target_path,"images")):
        if file.endswith(".jpeg"):
            report["images"]["count"] += 1

    return report

def clear_runr_dir(dir):
    # Clean left over runs dir
    if os.path.isdir(dir):
        shutil.rmtree("./runs")

# endregion


In [5]:
# Define paths
from pathlib import Path

ROOT_DIR = Path().resolve() # Root path

LABEL_IMAGE_PATH = Path(ROOT_DIR, "data_label_image")
DATASET_PATH = Path(ROOT_DIR, "datasets")
TRAIN_DS_PATH = Path(DATASET_PATH, "train")
TEST_DS_PATH = Path(DATASET_PATH, "test")
VAL_DS_PATH = Path(DATASET_PATH, "val")

TEST_SIZE = 0.15 # % of test size
VAL_SIZE = 0.15 # % of test size

# establish datasets directory
create_datasets_dir(DATASET_PATH)

# get unique list of file
fn_list = os.listdir(LABEL_IMAGE_PATH)
fn_unique_list = list(set(["".join(fn.split(".")[:-1]) for fn in fn_list]))

# split train and test dataset
fn_train, fn_test = train_test_split(
    fn_unique_list, test_size=TEST_SIZE, random_state=42)


train_val, fn_test = train_test_split(fn_unique_list, test_size=TEST_SIZE, random_state=42) # test % = 15%

fn_train, fn_val = train_test_split(train_val, test_size=TEST_SIZE/(1-TEST_SIZE), random_state=42) # (1-15%) x size = 15%

# # clone files to destination
copy_files(fn_train, LABEL_IMAGE_PATH,  TRAIN_DS_PATH)
copy_files(fn_test, LABEL_IMAGE_PATH,  TEST_DS_PATH)
copy_files(fn_val, LABEL_IMAGE_PATH,  VAL_DS_PATH)

# train_report
train_report = report_info(TRAIN_DS_PATH)
print(
    f"\nReport: Train Dataset\nImages Path: {train_report['images']['path']}\nImages Count: {train_report['images']['count']}\nLabel Path: {train_report['labels']['path']}\nLabels Count: {train_report['labels']['count']}")

# val_report
val_report = report_info(VAL_DS_PATH)
print(
    f"\nReport: Val Dataset\nImages Path: {val_report['images']['path']}\nImages Count: {val_report['images']['count']}\nLabel Path: {val_report['labels']['path']}\nLabels Count: {val_report['labels']['count']}")

# test_report
test_report = report_info(TEST_DS_PATH)
print(
    f"\nReport: Test Dataset\nImages Path: {test_report['images']['path']}\nImages Count: {test_report['images']['count']}\nLabel Path: {test_report['labels']['path']}\nLabels Count: {test_report['labels']['count']}")

clear_runr_dir("./runs")



Report: Train Dataset
Images Path: /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/train/images
Images Count: 378
Label Path: /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/train/labels
Labels Count: 378

Report: Val Dataset
Images Path: /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/val/images
Images Count: 84
Label Path: /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/val/labels
Labels Count: 84

Report: Test Dataset
Images Path: /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/test/images
Images Count: 81
Label Path: /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/test/labels
Labels Count: 81


---

## Train YOLOv8 Model

### Create configuration yaml files

- YAML file in `datasets` directory

```yaml
names:
  0: <label_name>
train: <absolute path of train dataset's images>
val: <absolute path of val dataset's images>
```


In [6]:
import yaml

YAML_PATH = "datasets/config.yaml"  # path to data file

data = dict(
    train = Path(TRAIN_DS_PATH, "images").absolute().as_posix(),
    val = Path(TRAIN_DS_PATH, "images").absolute().as_posix(),
    names = {0: "car_plate"}
)

with open(YAML_PATH, 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

### Train Model

- Train custom model from scratch.


In [7]:
from ultralytics import YOLO

YOLO_V8N = "./model/base/yolov8n.pt"  # the model to train customized model

car_plate_model = YOLO(YOLO_V8N)     # Define model object from default v8n model
car_plate_model.train(data=YAML_PATH)    # Train Yolo model

Ultralytics YOLOv8.0.212 🚀 Python-3.11.5 torch-2.1.1 CPU (Apple M2)
engine/trainer: task=detect, mode=train, model=./model/base/yolov8n.pt, data=datasets/config.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, 

train: Scanning /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/train/labels... 378 images, 0 backgrounds, 0 corrupt: 100%|██████████| 378/378 [00:00<00:00, 1437.63it/s]

train: WARNING ⚠️ /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/train/images/Audi A3 Convertible with license plate_18.jpeg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/train/images/Audi A3 Convertible with license plate_30.jpeg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/train/images/Ford Focus with license plate_15.jpeg: corrupt JPEG restored and saved
train: New cache created: /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/train/labels.cache



val: Scanning /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/train/labels.cache... 378 images, 0 backgrounds, 0 corrupt: 100%|██████████| 378/378 [00:00<?, ?it/s]

train: WARNING ⚠️ /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/train/images/Audi A3 Convertible with license plate_18.jpeg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/train/images/Audi A3 Convertible with license plate_30.jpeg: corrupt JPEG restored and saved
train: WARNING ⚠️ /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/train/images/Ford Focus with license plate_15.jpeg: corrupt JPEG restored and saved


...

                   all        378        393      0.999          1      0.995      0.959



100 epochs completed in 5.889 hours.
Optimizer stripped from /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/runs/detect/train/weights/best.pt, 6.3MB

Validating /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.0.212 🚀 Python-3.11.5 torch-2.1.1 CPU (Apple M2)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   8%|▊         | 1/12 [00:03<00:38,  3.54s/it]Corrupt JPEG data: 1 extraneous bytes before marker 0xd9
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  17%|█▋        | 2/12 [00:07<00:38,  3.85s/it]Corrupt JPEG data: premature end of data segment
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  25%|██▌       | 3/12 [00:12<00:38,  4.25s/it]Corrupt JPEG data: premature end of data segment
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:57<00:00,  4.76s/it]


                   all        378        393      0.999          1      0.995      0.959
Speed: 0.9ms preprocess, 133.8ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x28a8ad010>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,    

## Predict using Custom Model


In [4]:
from ultralytics import YOLO
from pathlib import Path

CAR_PLATE_MODEL_PATH = "./model/car_plate/best.pt"
TEST_IMAGE_PATH = Path(".","datasets","test","images").absolute().as_posix()

# Load a model
car_plate_model = YOLO(CAR_PLATE_MODEL_PATH)  # pretrained YOLOv8n model

car_plate_model.predict(TEST_IMAGE_PATH, save=True, imgsz=320, conf=0.5)


image 1/81 /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/test/images/Audi A3 Convertible with license plate_39.jpeg: 224x320 1 car_plate, 40.7ms
image 2/81 /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/test/images/Audi A3 Convertible with license plate_4.jpeg: 192x320 1 car_plate, 14.5ms
image 3/81 /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/test/images/Audi A3 Convertible with license plate_46.jpeg: 224x320 1 car_plate, 20.9ms
image 4/81 /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/test/images/Audi A3 Convertible with license plate_53.jpeg: 224x320 1 car_plate, 16.6ms
image 5/81 /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/test/images/Audi A4 with license plate_16.jpeg: 192x320 1 car_plate, 14.3ms
image 6/81 /Users/simon/Documents/Github/ITE-5310-Machine-Learning-Assignment10/datasets/test/images/Audi A5 with license plate

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'car_plate'}
 orig_img: array([[[ 38,  37,  27],
         [ 42,  43,  33],
         [ 42,  43,  34],
         ...,
         [ 41,  67,  54],
         [ 39,  65,  51],
         [ 18,  47,  32]],
 
        [[ 39,  38,  28],
         [ 41,  42,  32],
         [ 37,  40,  31],
         ...,
         [ 58,  83,  69],
         [ 59,  88,  73],
         [ 41,  70,  55]],
 
        [[ 38,  39,  30],
         [ 38,  41,  32],
         [ 33,  37,  31],
         ...,
         [ 57,  82,  68],
         [ 64,  93,  78],
         [ 51,  83,  66]],
 
        ...,
 
        [[117, 126, 135],
         [115, 124, 133],
         [122, 131, 140],
         ...,
         [111, 120, 129],
         [114, 123, 132],
         [108, 117, 126]],
 
        [[123, 132, 141],
         [124, 133, 142],
         [127, 136, 145],
         ...,
         [106, 115, 124],
